In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=2, style="whitegrid", rc={'figure.figsize' : (15,7)})
import matplotlib.pyplot as plt
from scipy import special
plt.style.use('dark_background')

Готовим данные для последующего анализа в Tableau

In [2]:
df = pd.read_csv('https://drive.usercontent.google.com/download?id=1PvE55GbXpTPbnNw83DgbnsnIwfmt1NJn&export=download&authuser=0&confirm=t&uuid=741d50f7-0d0e-40e1-9a52-8a39ceb14f56&at=AENtkXaNvsEoTytPIm8_8GpT6N0f%3A1733144765054')

In [25]:
df.head(3)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal


In [26]:
# смотрим временной период данных
pd.DataFrame({'time range': [df['date'].min(),df['date'].max()]})

,time range
0,2020-01-01
1,2020-03-31


In [29]:
#сортируем по дате, для каждого device_id удаляем дубликаты даты, оставляя только запись с первой датой появления этого айди в данных
mindate = df.sort_values('date').drop_duplicates('device_id')
mindate

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
13192,2020-01-01,tap_basket,NaN,android,17289661,male,Saint-Petersburg,vk_ads
13191,2020-01-01,tap_basket,NaN,android,12215118,male,Saint-Petersburg,facebook_ads
13190,2020-01-01,tap_basket,NaN,ios,9163079,male,Moscow,vk_ads
13189,2020-01-01,tap_basket,NaN,android,1948894,female,Saint-Petersburg,google_ads
...,...,...,...,...,...,...,...,...
2497656,2020-03-31,app_start,NaN,ios,5682834,female,Moscow,-
2499467,2020-03-31,app_start,NaN,android,9339041,male,Saint-Petersburg,-
2499780,2020-03-31,app_start,NaN,ios,32346202,female,Moscow,yandex-direct
2498997,2020-03-31,app_start,NaN,ios,23772077,male,Moscow,vk_ads


In [30]:
mindate = mindate[['date','device_id']]
mindate.columns = ['dt_of_appearance','device_id']
mindate

,dt_of_appearance,device_id
0,2020-01-01,669460
13192,2020-01-01,17289661
13191,2020-01-01,12215118
13190,2020-01-01,9163079
13189,2020-01-01,1948894
...,...,...
2497656,2020-03-31,5682834
2499467,2020-03-31,9339041
2499780,2020-03-31,32346202
2498997,2020-03-31,23772077


In [33]:
# Для каждой записи с device_id вписываем в колонку дату появления этого айди в данных
df = df.merge(mindate, how='left', on='device_id')
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,dt_of_appearance
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06


Теперь с добавленной когортой dt_of_appearance для каждого действия можно считать что угодно